In [1]:
import cv2
import numpy as np
from PIL import Image
import pandas
import math
# from generate_inputs import GenerateInputVector
from generate_inputs import device
from generate_inputs import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

In [2]:
# Load data
def GenerateInputVector(traj_num_list):
    max_succ_dist = 0.5
    max_succ_ang = math.radians(10)
    
    image_list = []
    image1_list = []
    image2_list = []
    rel_pose_list = []
    
    for k in traj_num_list:
        dir = './data/traj_' + str(k) + '/'
        print(dir)
        pose_list = pandas.read_csv(dir + 'pose_list.csv', header=None).values
        N = len(pose_list)
        for i in range(0, N-1):
            image_list.append(cv2.imread(dir + 'img_' + str(i) + '.jpg', 0)/255)
            for j in range(i+1, N, 1):
                rel_pose = RelativePose(pose_list[i], pose_list[j])
                dist = np.sqrt(rel_pose[0]*rel_pose[0] + rel_pose[1]*rel_pose[1])
                ang = rel_pose[2]
                if dist < max_succ_dist and ang < max_succ_ang:
                    image1_list.append(cv2.imread(dir + 'img_' + str(i) + '.jpg', 0)/255)
                    image2_list.append(cv2.imread(dir + 'img_' + str(j) + '.jpg', 0)/255)
                    rel_pose_list.append(rel_pose)
        image_list.append(cv2.imread(dir + 'img_' + str(N-1) + '.jpg', 0)/255)
    return pose_list, image_list, image1_list, image2_list, rel_pose_list
traj_num_list = np.arange(2,3)
pose_list, image_list, image1_list, image2_list, rel_pose_list = GenerateInputVector(traj_num_list)

./data/traj_2/


In [4]:
len(image1_list)

13632

In [ ]:
dataset_dict = {}
dataset_dict['']
np.save()

In [ ]:
# Verifying input dimensions

device = torch.device("cuda")
net = Net()
net = net.to(device)
print(net)

input = torch.randn(1, 2, 224, 224).to(device)
out = net(input)
print(out.shape)

x = np.stack([image1_list[0], image2_list[0]])
x = np.expand_dims(x, axis=0)
print(x.shape)

In [ ]:
# Training X- network

import random
import torch.optim as optim
idxs = list(range(len(image1_list)))
random.shuffle(idxs)
net_x = Net()
net_x = net_x.to(device)
optimizer = optim.Adam(net_x.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(image1_list) * 0.8)]
test_idxs = idxs[int(len(image1_list) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(rel_pose_list[idx][0]).float().to(device)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_x, "net_x_2.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(rel_pose_list[idx][0]).float().to(device)
        #print(out.shape, target.shape)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        optimizer.zero_grad()
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


In [ ]:
# Training Y- network

import random
import torch.optim as optim
idxs = list(range(len(image1_list)))
random.shuffle(idxs)
net_y = Net()
optimizer = optim.Adam(net_y.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(image1_list) * 0.8)]
test_idxs = idxs[int(len(image1_list) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(rel_pose_list[idx][1]).float().cuda()
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_y, "net_y_2.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(rel_pose_list[idx][1]).float().cuda()
        #criterion = nn.MSELoss()
        optimizer.zero_grad()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


In [ ]:
# Training Angle- network

import random
import torch.optim as optim
idxs = list(range(len(image1_list)))
random.shuffle(idxs)
net_angle = Net()
optimizer = optim.Adam(net_angle.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(image1_list) * 0.8)]
test_idxs = idxs[int(len(image1_list) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(rel_pose_list[idx][2]).float().cuda()
        criterion = nn.MSELoss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_angle, "net_angle_2.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(rel_pose_list[idx][2]).float().cuda()
        optimizer.zero_grad()
        criterion = nn.MSELoss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))
